<a href="https://colab.research.google.com/github/Thisun1997/spark_test/blob/main/sparkTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
print(os.listdir('./sample_data'))

['anscombe.json', 'README.md', 'mnist_test.csv', 'california_housing_test.csv', 'california_housing_train.csv', 'mnist_train_small.csv']


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')

--2020-10-28 16:01:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.0.74.51, 34.205.238.171, 34.193.208.150, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.0.74.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.13M  17.9MB/s    in 0.7s    

2020-10-28 16:01:43 (17.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [ ]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://45cc5721e071.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":1,"gauge":0,"rate1":0.00003087011809866623,"rate5":0.00094710155304635,"rate15":0.0007304586585316237,"p50":30084441177,"p90":30084441177,"p95":30084441177,"p99":30084441177},"http":{"count":1,"rate1":0.00001872367309579094,"rate5":0.000856972923876307,"rate15":0.0007065113752682741,"p50":79932596,"p90":79932596,"p95":79932596,"p99":79932596}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://45cc5721e071.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":34,"gauge":0,"rate1":0.015610128001601617,"rate5":0.024577436694581643,"rate15":0.021311124000104275,"p50":679000105,"p90":62111033589,"p95":100307398560.5,"p99":116016234608},"http":{"count":97,"rate1":0.022736049892

In [ ]:
rdd = sc.textFile('/content/spark-2.4.7-bin-hadoop2.7/python/test_support/sql/TestData/data.txt')

In [ ]:
rdd.collect()

['1999\t1999', '2000\t2000', '2001\t2001', '2010\t2010', '2018\t2018']

In [ ]:
df = rdd.map(lambda x: x.split("\t")).toDF(["TEST","TEST_PART"])

In [ ]:
df.show()

+----+---------+
|TEST|TEST_PART|
+----+---------+
|1999|     1999|
|2000|     2000|
|2001|     2001|
|2010|     2010|
|2018|     2018|
+----+---------+



In [ ]:
df.printSchema()

root
 |-- TEST: string (nullable = true)
 |-- TEST_PART: string (nullable = true)



In [ ]:
from pyspark.sql.types import DateType
df1 = df.withColumn("TEST_COL", df['TEST'].cast('long')).drop('TEST').withColumn('TEST_PART_COL', df['TEST_PART'].cast('long')).drop('TEST_PART')
df1.show()
print(df1.schema)

+--------+-------------+
|TEST_COL|TEST_PART_COL|
+--------+-------------+
|    1999|         1999|
|    2000|         2000|
|    2001|         2001|
|    2010|         2010|
|    2018|         2018|
+--------+-------------+

StructType(List(StructField(TEST_COL,LongType,true),StructField(TEST_PART_COL,LongType,true)))


In [ ]:
df1.printSchema()

root
 |-- TEST_COL: long (nullable = true)
 |-- TEST_PART_COL: long (nullable = true)



In [ ]:
df1.write.format("parquet").partitionBy("TEST_PART_COL").option("path", "/content/spark-2.4.7-bin-hadoop2.7/python/test_support/sql/TestData/parquet").saveAsTable("TEST_TABLE")


In [ ]:
spark.sql("SELECT * FROM TEST_TABLE WHERE TEST_PART_COL = TEST_COL AND (TEST_COL = 2001 OR TEST_COL = 1999)").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter (('TEST_PART_COL = 'TEST_COL) && (('TEST_COL = 2001) || ('TEST_COL = 1999)))
   +- 'UnresolvedRelation `TEST_TABLE`

== Analyzed Logical Plan ==
TEST_COL: bigint, TEST_PART_COL: bigint
Project [TEST_COL#32L, TEST_PART_COL#33L]
+- Filter ((TEST_PART_COL#33L = TEST_COL#32L) && ((TEST_COL#32L = cast(2001 as bigint)) || (TEST_COL#32L = cast(1999 as bigint))))
   +- SubqueryAlias `default`.`test_table`
      +- Relation[TEST_COL#32L,TEST_PART_COL#33L] parquet

== Optimized Logical Plan ==
Project [TEST_COL#32L, TEST_PART_COL#33L]
+- Filter ((((((TEST_PART_COL#33L = 2001) || (TEST_PART_COL#33L = 1999)) && isnotnull(TEST_PART_COL#33L)) && isnotnull(TEST_COL#32L)) && (TEST_PART_COL#33L = TEST_COL#32L)) && ((TEST_COL#32L = 2001) || (TEST_COL#32L = 1999)))
   +- Relation[TEST_COL#32L,TEST_PART_COL#33L] parquet

== Physical Plan ==
*(1) Project [TEST_COL#32L, TEST_PART_COL#33L]
+- *(1) Filter ((isnotnull(TEST_COL#32L) && (TEST_PART_COL#33L = TEST_

In [ ]:
spark.sql("SELECT * FROM TEST_TABLE WHERE TEST_PART_COL = TEST_COL AND (TEST_COL > 1999 AND TEST_COL <= 2001)").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter (('TEST_PART_COL = 'TEST_COL) && (('TEST_COL > 1999) && ('TEST_COL <= 2001)))
   +- 'UnresolvedRelation `TEST_TABLE`

== Analyzed Logical Plan ==
TEST_COL: bigint, TEST_PART_COL: bigint
Project [TEST_COL#32L, TEST_PART_COL#33L]
+- Filter ((TEST_PART_COL#33L = TEST_COL#32L) && ((TEST_COL#32L > cast(1999 as bigint)) && (TEST_COL#32L <= cast(2001 as bigint))))
   +- SubqueryAlias `default`.`test_table`
      +- Relation[TEST_COL#32L,TEST_PART_COL#33L] parquet

== Optimized Logical Plan ==
Project [TEST_COL#32L, TEST_PART_COL#33L]
+- Filter ((((((isnotnull(TEST_PART_COL#33L) && isnotnull(TEST_COL#32L)) && (TEST_PART_COL#33L <= 2001)) && (TEST_PART_COL#33L > 1999)) && (TEST_PART_COL#33L = TEST_COL#32L)) && (TEST_COL#32L > 1999)) && (TEST_COL#32L <= 2001))
   +- Relation[TEST_COL#32L,TEST_PART_COL#33L] parquet

== Physical Plan ==
*(1) Project [TEST_COL#32L, TEST_PART_COL#33L]
+- *(1) Filter (((isnotnull(TEST_COL#32L) && (TEST_PART_COL#33L = 

In [ ]:
spark.sql("SELECT * FROM TEST_TABLE WHERE TEST_PART_COL = ABS(TEST_COL) AND (TEST_COL = 2001 OR TEST_COL = 1999)").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter (('TEST_PART_COL = 'ABS('TEST_COL)) && (('TEST_COL = 2001) || ('TEST_COL = 1999)))
   +- 'UnresolvedRelation `TEST_TABLE`

== Analyzed Logical Plan ==
TEST_COL: bigint, TEST_PART_COL: bigint
Project [TEST_COL#32L, TEST_PART_COL#33L]
+- Filter ((TEST_PART_COL#33L = abs(TEST_COL#32L)) && ((TEST_COL#32L = cast(2001 as bigint)) || (TEST_COL#32L = cast(1999 as bigint))))
   +- SubqueryAlias `default`.`test_table`
      +- Relation[TEST_COL#32L,TEST_PART_COL#33L] parquet

== Optimized Logical Plan ==
Project [TEST_COL#32L, TEST_PART_COL#33L]
+- Filter (((isnotnull(TEST_PART_COL#33L) && isnotnull(TEST_COL#32L)) && (TEST_PART_COL#33L = abs(TEST_COL#32L))) && ((TEST_COL#32L = 2001) || (TEST_COL#32L = 1999)))
   +- Relation[TEST_COL#32L,TEST_PART_COL#33L] parquet

== Physical Plan ==
*(1) Project [TEST_COL#32L, TEST_PART_COL#33L]
+- *(1) Filter ((isnotnull(TEST_COL#32L) && (TEST_PART_COL#33L = abs(TEST_COL#32L))) && ((TEST_COL#32L = 2001) || (TES

In [ ]:
spark.sql("SELECT * FROM TEST_TABLE WHERE TEST_PART_COL = ABS(TEST_COL) AND (TEST_COL > 1999 AND TEST_COL <= 2001)").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter (('TEST_PART_COL = 'ABS('TEST_COL)) && (('TEST_COL > 1999) && ('TEST_COL <= 2001)))
   +- 'UnresolvedRelation `TEST_TABLE`

== Analyzed Logical Plan ==
TEST_COL: bigint, TEST_PART_COL: bigint
Project [TEST_COL#32L, TEST_PART_COL#33L]
+- Filter ((TEST_PART_COL#33L = abs(TEST_COL#32L)) && ((TEST_COL#32L > cast(1999 as bigint)) && (TEST_COL#32L <= cast(2001 as bigint))))
   +- SubqueryAlias `default`.`test_table`
      +- Relation[TEST_COL#32L,TEST_PART_COL#33L] parquet

== Optimized Logical Plan ==
Project [TEST_COL#32L, TEST_PART_COL#33L]
+- Filter ((((isnotnull(TEST_PART_COL#33L) && isnotnull(TEST_COL#32L)) && (TEST_PART_COL#33L = abs(TEST_COL#32L))) && (TEST_COL#32L > 1999)) && (TEST_COL#32L <= 2001))
   +- Relation[TEST_COL#32L,TEST_PART_COL#33L] parquet

== Physical Plan ==
*(1) Project [TEST_COL#32L, TEST_PART_COL#33L]
+- *(1) Filter (((isnotnull(TEST_COL#32L) && (TEST_PART_COL#33L = abs(TEST_COL#32L))) && (TEST_COL#32L > 1999)) && 